In [ ]:
import subprocess
from subprocess import PIPE
import rasterio
import json
import glob
import pandas as pd
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.model_selection import train_test_split
import shap
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [ ]:
path_labels = 'path'
path_training_images = 'path'
path_testing_images = 'path'
def image_label(product_id):
    rootdir = "path"
    pd = product_id.split("_")
    pd = pd[3] + "_" + pd[4] + "_" + pd[5] + "_" + pd[6]    
    json_data=open(rootdir + pd +"/stac.json", "rb")
    jdata = json.load(json_data)
    flood = jdata["properties"]["FLOODING"]
    if (flood == "False"):
        image_label = 0
    else:
        image_label = 1
    
    return image_label

In [ ]:
def product_name(path):
    
    json_data=open(path+"/stac.json", "rb")
    jdata = json.load(json_data)
    
    return  jdata['id']
    def load_data():
    
    data = [] 
    images = []
    labels = []
    
    for folder in imagelist:
        try:
            product_id = product_name(folder)
            print(f"Processing {product_id} image product")
            label = image_label(product_id)
        
            # Open the img
            image = cv2.imread(folder + "/stack.tif")
            # Append the image and its corresponding label to the output
            images.append(image)
            labels.append(label)
        except:
            pass
        
        
    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')
        
        
    data.append([images, labels])     

    return images, labels
    imagelist = []
rootdir = path_training_images
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        imagelist.append(d)
        
        
print(f"The number of training samples is currently = {len(imagelist)}")
    

In [ ]:
train_images, train_labels = load_data()

In [ ]:
def product_name(path):
    
    json_data=open(path+"/stac.json", "rb")
    jdata = json.load(json_data)
    
    return  jdata['id']
    def load_data():
    
    data = [] 
    images = []
    labels = []
    
    for folder in imagelist:
        try:
            product_id = product_name(folder)
            print(f"Processing {product_id} image product")
            label = image_label(product_id)
        
            # Open the img
            image = cv2.imread(folder + "/stack.tif")
            # Append the image and its corresponding label to the output
            images.append(image)
            labels.append(label)
        except:
            pass
        
        
    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')
        
        
    data.append([images, labels])     

    return images, labels
    imagelist = []
rootdir = path_testing_images
for file in os.listdir(rootdir):
    d = os.path.join(rootdir, file)
    if os.path.isdir(d):
        imagelist.append(d)
        
        
print(f"The number of testing samples is currently = {len(imagelist)}")

In [ ]:
test_images, test_labels = load_data()

In [ ]:
class FloodDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.tensor(images).permute(0, 3, 1, 2)
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# DataLoaders
train_dataset = FloodDataset(train_images, train_labels)
test_dataset = FloodDataset(test_images, test_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Swish activation
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# Hierarchical Multi-Attention block
class HierarchicalMultiAttention(nn.Module):
    def __init__(self, in_dim, key_dim=64):
        super(HierarchicalMultiAttention, self).__init__()
        self.W_q = nn.Linear(in_dim, key_dim)
        self.W_k = nn.Linear(in_dim, key_dim)
        self.W_v = nn.Linear(in_dim, key_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        d_k = K.size(-1)
        attn = self.softmax(Q @ K.transpose(-2, -1) / np.sqrt(d_k))
        return attn @ V

# HMA-EfficientNetV2 model
class HMAEfficientNetV2(nn.Module):
    def __init__(self, num_classes=2):
        super(HMAEfficientNetV2, self).__init__()
        base_model = models.efficientnet_v2_s(weights='DEFAULT')
        self.features = base_model.features
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.swish = Swish()
        self.hma_blocks = nn.ModuleList([
            HierarchicalMultiAttention(1280),
            HierarchicalMultiAttention(1280),
            HierarchicalMultiAttention(1280)
        ])
        self.concat_fc = nn.Linear(3 * 64, 128)
        nn.init.kaiming_uniform_(self.concat_fc.weight, nonlinearity='relu')
        nn.init.zeros_(self.concat_fc.bias)
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x).squeeze(-1).squeeze(-1)
        features = []
        for hma in self.hma_blocks:
            hma_out = hma(x.unsqueeze(1))
            features.append(hma_out.squeeze(1))
        hma_cat = torch.cat(features, dim=-1)
        fhma = self.swish(self.concat_fc(hma_cat))
        out = self.fc(fhma)
        return out

model = HMAEfficientNetV2()

# Loss function with label smoothing and regularization
def custom_loss(y_pred, y_true, smoothing=0.1, alpha=0.9, reg_lambda=1e-4):
    num_classes = y_pred.size(1)
    y_true = F.one_hot(y_true, num_classes).float()
    y_smooth = (1 - smoothing) * y_true + smoothing / num_classes
    ce_loss = -torch.sum(y_smooth * F.log_softmax(y_pred, dim=1), dim=1).mean()
    l2_reg = sum(torch.norm(p)**2 for p in model.parameters())
    return alpha * ce_loss + (1 - alpha) * reg_lambda * l2_reg

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss = custom_loss(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss/len(train_loader):.4f}")

all_preds = []
all_labels = []
model.eval()
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, all_preds)

print("\nModel Performance:")
print(f"Precision (%): {100 * precision:.1f}")
print(f"Recall (%):    {100 * recall:.1f}")
print(f"Accuracy (%):  {100 * accuracy:.1f}")


In [ ]:
# SHAP analysis
background = next(iter(DataLoader(train_dataset, batch_size=100, shuffle=True)))[0]
class WrappedModel(nn.Module):
    def __init__(self, model):
        super(WrappedModel, self).__init__()
        self.model = model
    def forward(self, x):
        with torch.no_grad():
            return F.softmax(self.model(x), dim=1)
wrapped_model = WrappedModel(model)
explainer = shap.DeepExplainer(wrapped_model, background)
sample_inputs, _ = next(iter(DataLoader(test_dataset, batch_size=10, shuffle=False)))
shap_values = explainer.shap_values(sample_inputs)
mean_shap_per_channel = np.abs(shap_values[1]).mean(axis=(0, 2, 3))
for i, val in enumerate(mean_shap_per_channel):
    print(f"Channel {i}: Mean SHAP Value = {val:.3f}")